In [1]:
%cd nanoGPT
!python data/shakespeare_char/prepare.py

/usr/local/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/app/repos/selfishTokens_2/nanoGPT
length of dataset in characters: 1,115,394
all the unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65
train has 1,003,854 tokens
val has 111,540 tokens


In [2]:
import subprocess
import sys

def run_training(args):
    """
    Runs the training script with the given arguments and streams the output in real time in Jupyter Notebook.
    
    Args:
        args (dict): Dictionary of command-line arguments where keys are the argument names 
                     and values are the argument values.
    """
    # Base command with the script and config file
    command = [
        "python", "train.py", "config/train_shakespeare_char.py"
    ] + [f'--{k}={v}' for k, v in args.items()]

    # Use Popen to continuously capture and stream the output
    with subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, bufsize=1) as process:
        for line in process.stdout:
            print(line, end="")  # Print the output as it arrives
            sys.stdout.flush()  # Ensure immediate flush of the output in Jupyter
        
        # Wait for the process to complete and handle errors if they occur
        process.wait()  # Ensure the process completes
        if process.returncode != 0:
            # In case of errors, print the stderr
            error_output = process.stderr.read()
            print("Error:", error_output)

In [8]:
#baseline
args = {
    "out_dir": "out/baseline"
}
run_training(args)


Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

In [12]:
#slo
args = {
    "out_dir": "out/slo",
    "log_interval": 2,
    "max_iters": 160,
    "lr_decay_iters": 160,
    "eval_interval": 5,
    "big_language_prob": 0.0,
    "warmup_iters": 2,
}
run_training(args)

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

In [13]:
for margin in [8, 4, 2, 1]:
    args = {
        "out_dir": f"out/proposed_margin_{margin}",
        "model_type": "proposed",
        "margin": float(margin)

    }
    run_training(args)
    print(f'done for margin={margin}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to


# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

warmup_iters = 100 # not super necessary potentially

# on macbook also add
# device = 'cpu'  # run on cpu only
# compile = False # do not torch compile the model

Overriding: out_dir = out/proposed_margin_4
Overriding: model_type = proposed
Overriding: margin = 4
tokens per iteration will be: 7

wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

warmup_iters = 100 # not super necessary potentially

# on macbook also add
# device = 'cpu'  # run on cpu only
# compile = False # do not torch compile the model

Overriding: out_dir = out/proposed_margin_2
Overriding: model_type = proposed
Overriding: margin = 2
tokens per iteration will be: 768
found vocab_size = 65 (inside data/shakespeare_char/meta.pkl)
Initializing a new model from scratch
number of parameters: 0.80M
num decayed parameter tensors: 18, with 811,264 parameters
num non-decay

block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

warmup_iters = 100 # not super necessary potentially

# on macbook also add
# device = 'cpu'  # run on cpu only
# compile = False # do not torch compile the model

Overriding: out_dir = out/proposed_margin_1
Overriding: model_type = proposed
Overriding: margin = 1
tokens per iteration will be: 768
found vocab_size = 65 (inside data/shakespeare_char/meta.pkl)
Initializing a new model from scratch
number of parameters: 0.80M
num decayed parameter tensors: 18, with 811,264 parameters
num non-decayed parameter tensors: 9, with 1,152 parameters
using fused AdamW: False
step 0: train loss 4.8628, val los

In [44]:
# agg
args = {
    "out_dir": f"out/agg_0.02_1600",
    "model_type": "agg",
    "agg_alpha": 0.02,
    "agg_k": 1600
}
run_training(args)

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

In [46]:
# agg
args = {
    "out_dir": f"out/agg_0.2_1600",
    "model_type": "agg",
    "agg_alpha": 0.2,
    "agg_k": 1600
}
run_training(args)

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

In [47]:
# cosreg
args = {
    "out_dir": f"out/cosreg_1",
    "model_type": "cosreg",
    "cosreg_gamma": 1

}
run_training(args)

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

In [18]:
args = {
    "out_dir": f"out/adv_0.05",
    "model_type": "adv",
    "adv_alpha": 0.005

}
run_training(args)

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

In [62]:
margin = 1
temperature = 0.46
args = {
    "out_dir": f"out/proposed_margin_{margin}_{T}",
    "model_type": "proposed",
    "margin": float(margin),
    "temperature": temperature
}
run_training(args)
print(f'done for margin={margin}; T={T}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

In [64]:
margin = 0.5
args = {
    "out_dir": f"out/proposed_margin_{margin}",
    "model_type": "proposed",
    "margin": margin
}
run_training(args)
print(f'done for margin={margin}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

In [65]:
args = {
    "out_dir": f"out/cwt",
    "model_type": "cwt",
}
run_training(args)
print(f'done for cwt')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

In [67]:
margin = 0.0
args = {
    "out_dir": f"out/proposed_margin_{margin}",
    "model_type": "proposed",
    "margin": margin
}
run_training(args)
print(f'done for margin={margin}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

In [77]:
#baseline logged
args = {
    "out_dir": "out/baseline_logged",
    "log_output_loss": "True"
}
run_training(args)


Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

In [78]:
for margin in [2.0]:
    args = {
        "out_dir": f"out/proposed_margin_{margin}_logged",
        "model_type": "proposed",
        "margin": margin,
        "log_output_loss": "True"
    }
    run_training(args)
    print(f'done for margin={margin}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

In [79]:
#baseline wo weight_tying
args = {
    "out_dir": "out/baseline_wo_weight_tying",
    "weight_tying": "False"
}
run_training(args)


Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

In [80]:
for margin in [1.0, 2.0]:
    args = {
        "out_dir": f"out/proposed_margin_{margin}_wo_weight_tying",
        "model_type": "proposed",
        "margin": margin,
    "weight_tying": "False"
        
    }
    run_training(args)
    print(f'done for margin={margin}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

iter 7500: loss 1.2681, time 2155.44ms, mfu 0.01%
iter 7600: loss 1.2169, time 44.11ms, mfu 0.01%
iter 7700: loss 1.3002, time 187.94ms, mfu 0.01%
step 7750: train loss 2.1564, val loss 2.2718
iter 7800: loss 1.2005, time 77.79ms, mfu 0.01%
iter 7900: loss 1.2238, time 86.51ms, mfu 0.01%
step 8000: train loss 2.1613, val loss 2.2572
iter 8000: loss 1.1454, time 1014.13ms, mfu 0.01%
done for margin=1.0
Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accum

iter 7300: loss 1.2987, time 124.75ms, mfu 0.01%
iter 7400: loss 1.2709, time 103.24ms, mfu 0.01%
step 7500: train loss 1.6354, val loss 1.7624
saving checkpoint to out/proposed_margin_2.0_wo_weight_tying
iter 7500: loss 1.3706, time 952.79ms, mfu 0.01%
iter 7600: loss 1.3447, time 131.77ms, mfu 0.01%
iter 7700: loss 1.4027, time 58.58ms, mfu 0.01%
step 7750: train loss 1.6281, val loss 1.7984
iter 7800: loss 1.2826, time 55.58ms, mfu 0.01%
iter 7900: loss 1.3168, time 58.12ms, mfu 0.01%
step 8000: train loss 1.6309, val loss 1.7704
iter 8000: loss 1.2439, time 1516.71ms, mfu 0.01%
done for margin=2.0


In [83]:
for margin in [1.0, 2.0]:
    args = {
        "out_dir": f"out/proposed_margin_{margin}_separated_embeddings",
        "model_type": "proposed",
        "margin": margin,
        "separated_embeddings": "True"
    }
    run_training(args)
    print(f'done for margin={margin}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

step 7750: train loss 2.1264, val loss 2.2368
saving checkpoint to out/proposed_margin_1.0_separated_embeddings
iter 7800: loss 1.2721, time 100.54ms, mfu 0.01%
iter 7900: loss 1.2210, time 89.21ms, mfu 0.01%
step 8000: train loss 2.1067, val loss 2.2398
iter 8000: loss 1.2777, time 1698.93ms, mfu 0.01%
done for margin=1.0
Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous

iter 7100: loss 1.3753, time 106.14ms, mfu 0.01%
iter 7200: loss 1.2651, time 140.58ms, mfu 0.01%
step 7250: train loss 1.6273, val loss 1.7867
saving checkpoint to out/proposed_margin_2.0_separated_embeddings
iter 7300: loss 1.3721, time 323.87ms, mfu 0.01%
iter 7400: loss 1.3809, time 111.44ms, mfu 0.01%
step 7500: train loss 1.6153, val loss 1.7770
saving checkpoint to out/proposed_margin_2.0_separated_embeddings
iter 7500: loss 1.3317, time 1953.87ms, mfu 0.01%
iter 7600: loss 1.3200, time 116.14ms, mfu 0.01%
iter 7700: loss 1.2579, time 130.29ms, mfu 0.01%
step 7750: train loss 1.6349, val loss 1.7921
iter 7800: loss 1.3961, time 92.00ms, mfu 0.01%
iter 7900: loss 1.3337, time 89.16ms, mfu 0.01%
step 8000: train loss 1.6162, val loss 1.7917
iter 8000: loss 1.3627, time 1587.85ms, mfu 0.01%
done for margin=2.0


In [87]:
# workaround to save embeddings in old format

import torch
for margin in [1.0, 2.0]:

    # Path to your original checkpoint
    ckpt_path = f"out/proposed_margin_{margin}_separated_embeddings/ckpt.pt"
    # Path to save the fixed checkpoint

    # Load the checkpoint
    checkpoint = torch.load(ckpt_path, map_location='cpu')

    # Assuming the structure of the list of weights
    # is in transformer.wte.weights.X and lm_head.weights.X
    # Stack the list of weights for transformer.wte
    wte_weight_list = []
    for i in range(130):  # Assuming 130 vectors
        wte_weight_list.append(checkpoint['model']['transformer.wte.weights.{}'.format(i)])
    wte_stacked = torch.stack(wte_weight_list)  # Shape: (130, 128)

    # Stack the list of weights for lm_head
    lm_head_weight_list = []
    for i in range(130):  # Assuming 130 vectors
        lm_head_weight_list.append(checkpoint['model']['lm_head.weights.{}'.format(i)])
    lm_head_stacked = torch.stack(lm_head_weight_list)  # Shape: (130, 128)

    # Now replace the old list of weights with the stacked versions
    checkpoint['model']['transformer.wte.weight'] = wte_stacked
    checkpoint['model']['lm_head.weight'] = lm_head_stacked

    # Optionally, delete the old entries to avoid confusion
    for i in range(130):
        del checkpoint['model']['transformer.wte.weights.{}'.format(i)]
        del checkpoint['model']['lm_head.weights.{}'.format(i)]

    # Save the updated checkpoint
    fixed_ckpt_path = f"out/proposed_margin_{margin}_separated_embeddings/ckpt_fixed.pt"
    torch.save(checkpoint, fixed_ckpt_path)

    print(f"Fixed checkpoint saved to {fixed_ckpt_path}")


/tmp/ipykernel_757/3992427102.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location='cpu')


Fixed checkpoint saved to out/proposed_margin_1.0_separated_embeddings/ckpt_fixed.pt
Fixed checkpoint saved to out/proposed_margin_2.0_separated_embeddings/ckpt_fixed.pt


In [89]:
!mv out/proposed_margin_1.0_separated_embeddings/ckpt.pt out/proposed_margin_1.0_separated_embeddings/ckpt_original.pt
!mv out/proposed_margin_2.0_separated_embeddings/ckpt.pt out/proposed_margin_2.0_separated_embeddings/ckpt_original.pt

!mv out/proposed_margin_1.0_separated_embeddings/ckpt_fixed.pt out/proposed_margin_1.0_separated_embeddings/ckpt.pt
!mv out/proposed_margin_2.0_separated_embeddings/ckpt_fixed.pt out/proposed_margin_2.0_separated_embeddings/ckpt.pt

In [91]:
for margin in [1.0, 2.0]:
    for softminus_c in [1.0, 5.0]:
        args = {
            "out_dir": f"out/softminus_margin_{margin}_{softminus_c}",
            "model_type": "proposed",
            "margin": margin,
            "softminus": "True",
            "softminus_c": softminus_c
        }
        run_training(args)
        print(f'done for softminus_c={softminus_c}, margin={margin}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

warmup_iters = 100 # not super necessary potentially

# on macbook also add
# device = 

iter 8000: loss 1.1937, time 975.81ms, mfu 0.01%
done for softminus_c=5.0, margin=1.0
Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1

iter 8000: loss 1.1782, time 1263.53ms, mfu 0.01%
done for softminus_c=1.0, margin=2.0
Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 

step 8000: train loss 2.5912, val loss 2.6705
saving checkpoint to out/softminus_margin_2.0_5.0
iter 8000: loss 1.2005, time 2032.09ms, mfu 0.01%
done for softminus_c=5.0, margin=2.0


In [92]:
for margin in [1.0, 2.0]:
    for softminus_c in [1.0, 5.0]:
        args = {
            "out_dir": f"out/softminus_margin_{margin}_{softminus_c}_threshold_detached",
            "model_type": "proposed",
            "margin": margin,
            "softminus": "True",
            "softminus_c": softminus_c,
            "detach_threshold": "True"
        }
        run_training(args)
        print(f'done for softminus_c={softminus_c}, margin={margin}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

warmup_iters = 100 # not super necessary potentially

# on macbook also add
# device = 'cpu'  # run on cpu only
# compile = False # do not torch compile the model

Overriding: out_dir = out/softminus_margin_1.0_5.0_threshold_detached
Overriding: model_type = proposed
Overriding: margin = 1.0
Overriding: softminus = True
Overriding: softminus_c = 5.0
Overriding: detach_threshold = True
tokens per iteration will be: 768
found vocab_size = 65 (inside data/shakespeare_char/meta.pkl)
Initializing a 


# on macbook also add
# device = 'cpu'  # run on cpu only
# compile = False # do not torch compile the model

Overriding: out_dir = out/softminus_margin_2.0_1.0_threshold_detached
Overriding: model_type = proposed
Overriding: margin = 2.0
Overriding: softminus = True
Overriding: softminus_c = 1.0
Overriding: detach_threshold = True
tokens per iteration will be: 768
found vocab_size = 65 (inside data/shakespeare_char/meta.pkl)
Initializing a new model from scratch
number of parameters: 0.80M
num decayed parameter tensors: 18, with 811,264 parameters
num non-decayed parameter tensors: 9, with 1,152 parameters
using fused AdamW: False
step 0: train loss 4.8628, val loss 4.8586
iter 0: loss 4.8597, time 1903.75ms, mfu -100.00%
iter 100: loss 2.6669, time 89.88ms, mfu 0.01%
iter 200: loss 2.5527, time 51.29ms, mfu 0.02%
step 250: train loss 3.2334, val loss 3.2420
saving checkpoint to out/softminus_margin_2.0_1.0_threshold_detached
iter 300: loss 2.6198, time 76.02ms, mfu 0.02%
iter 400: l

iter 0: loss 5.0013, time 1687.54ms, mfu -100.00%
iter 100: loss 2.5633, time 97.33ms, mfu 0.01%
iter 200: loss 2.3179, time 66.57ms, mfu 0.01%
step 250: train loss 3.6007, val loss 3.6088
saving checkpoint to out/softminus_margin_2.0_5.0_threshold_detached
iter 300: loss 2.2636, time 138.77ms, mfu 0.01%
iter 400: loss 2.2520, time 61.02ms, mfu 0.01%
step 500: train loss 3.5090, val loss 3.5247
saving checkpoint to out/softminus_margin_2.0_5.0_threshold_detached
iter 500: loss 2.1327, time 1013.00ms, mfu 0.01%
iter 600: loss 2.0582, time 95.17ms, mfu 0.01%
iter 700: loss 2.5116, time 89.00ms, mfu 0.01%
step 750: train loss 3.3361, val loss 3.3315
saving checkpoint to out/softminus_margin_2.0_5.0_threshold_detached
iter 800: loss 2.9160, time 95.81ms, mfu 0.01%
iter 900: loss 2.5690, time 50.34ms, mfu 0.01%
step 1000: train loss 3.4174, val loss 3.4212
iter 1000: loss 2.2182, time 1059.40ms, mfu 0.01%
iter 1100: loss 2.0493, time 64.82ms, mfu 0.01%
iter 1200: loss 1.9905, time 361.79ms,

In [96]:
for margin in [1.0, 2.0]:
    args = {
        "out_dir": f"out/proposed_margin_{margin}_detached_under_threshold",
        "model_type": "proposed",
        "margin": margin,
        "detach_logits_under_threshold": "True"
    }
    run_training(args)
    print(f'done for margin={margin}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

warmup_iters = 100 # not super necessary potentially

# on macbook also add
# device = 'cpu'  # run on cpu only
# compile = False # do not torch compile the model

Overriding: out_dir = out/proposed_margin_2.0_detached_under_threshold
Overriding: model_type = proposed
Overriding: margin = 2.0
Overriding: detach_logits_under_threshold = True
tokens per iteration will be: 768
found vocab_size = 65 (inside data/shakespeare_char/meta.pkl)
Initializing a new model from scratch
number of parameters: 0.80M
num decayed parameter tensors: 18, with 811,264 parameters
num non-decayed parameter tensors: 9, with 1,152 parameters
using fused AdamW: False
step 0: train loss 4.8628, val loss 4.8586
iter 0: loss 4.8688, time 1221.80ms, mfu -100.00%
iter 100: loss 2.8020, time 63.80ms, mfu 0.02%
iter 200: loss 2.5653, time 102.25ms, mfu 0.02%
step 250: train loss 2.5500, val loss 2.5394
saving checkpoint to out/proposed_margin_

In [95]:
for margin_by_weight_alpha in [32.0, 16.0, 8.0, 4.0, 2.0, 1.0, 0.5]:
    args = {
        "out_dir": f"out/proposed_margin_by_weight_alpha_{margin_by_weight_alpha}",
        "model_type": "proposed",
        "margin_by_weight": "1",
        "margin_by_weight_alpha": margin_by_weight_alpha
    }
    run_training(args)
    print(f'done for margin_by_weight_alpha={margin_by_weight_alpha}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

step 7750: train loss 1.4369, val loss 1.6754
iter 7800: loss 1.4041, time 78.97ms, mfu 0.01%
iter 7900: loss 1.3969, time 110.25ms, mfu 0.01%
step 8000: train loss 1.4624, val loss 1.6347
iter 8000: loss 1.3548, time 1462.78ms, mfu 0.01%
done for margin_by_weight_alpha=32.0
Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_he

step 7500: train loss 1.4384, val loss 1.6075
saving checkpoint to out/proposed_margin_by_weight_alpha_16.0
iter 7500: loss 1.4619, time 1822.87ms, mfu 0.01%
iter 7600: loss 1.4752, time 124.01ms, mfu 0.01%
iter 7700: loss 1.5124, time 114.71ms, mfu 0.01%
step 7750: train loss 1.4347, val loss 1.6648
iter 7800: loss 1.3820, time 129.72ms, mfu 0.01%
iter 7900: loss 1.4163, time 121.78ms, mfu 0.01%
step 8000: train loss 1.4521, val loss 1.6292
iter 8000: loss 1.3490, time 1543.89ms, mfu 0.01%
done for margin_by_weight_alpha=16.0
Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via comm

iter 7000: loss 1.4334, time 1298.76ms, mfu 0.01%
iter 7100: loss 1.5385, time 86.76ms, mfu 0.01%
iter 7200: loss 1.4509, time 158.14ms, mfu 0.01%
step 7250: train loss 1.4350, val loss 1.6329
iter 7300: loss 1.3745, time 146.12ms, mfu 0.01%
iter 7400: loss 1.3481, time 135.82ms, mfu 0.01%
step 7500: train loss 1.4318, val loss 1.6068
saving checkpoint to out/proposed_margin_by_weight_alpha_8.0
iter 7500: loss 1.4349, time 1657.96ms, mfu 0.01%
iter 7600: loss 1.4438, time 117.08ms, mfu 0.01%
iter 7700: loss 1.4993, time 107.02ms, mfu 0.01%
step 7750: train loss 1.4256, val loss 1.6549
iter 7800: loss 1.3709, time 101.55ms, mfu 0.01%
iter 7900: loss 1.4011, time 109.33ms, mfu 0.01%
step 8000: train loss 1.4457, val loss 1.6204
iter 8000: loss 1.3396, time 1444.56ms, mfu 0.01%
done for margin_by_weight_alpha=8.0
Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'o

iter 6700: loss 1.3350, time 95.64ms, mfu 0.01%
step 6750: train loss 1.4986, val loss 1.6684
iter 6800: loss 1.4039, time 96.93ms, mfu 0.01%
iter 6900: loss 1.4793, time 108.15ms, mfu 0.01%
step 7000: train loss 1.4702, val loss 1.6589
iter 7000: loss 1.3843, time 1473.63ms, mfu 0.01%
iter 7100: loss 1.4649, time 125.56ms, mfu 0.01%
iter 7200: loss 1.3746, time 140.46ms, mfu 0.01%
step 7250: train loss 1.4911, val loss 1.6523
saving checkpoint to out/proposed_margin_by_weight_alpha_4.0
iter 7300: loss 1.3102, time 178.61ms, mfu 0.01%
iter 7400: loss 1.3010, time 124.14ms, mfu 0.01%
step 7500: train loss 1.4904, val loss 1.6373
saving checkpoint to out/proposed_margin_by_weight_alpha_4.0
iter 7500: loss 1.3829, time 1488.58ms, mfu 0.01%
iter 7600: loss 1.3751, time 84.93ms, mfu 0.01%
iter 7700: loss 1.4511, time 93.62ms, mfu 0.01%
step 7750: train loss 1.4843, val loss 1.6812
iter 7800: loss 1.3277, time 165.13ms, mfu 0.01%
iter 7900: loss 1.3554, time 130.49ms, mfu 0.01%
step 8000: tr

iter 6000: loss 1.3153, time 1680.12ms, mfu 0.01%
iter 6100: loss 1.3747, time 128.19ms, mfu 0.01%
iter 6200: loss 1.1916, time 143.30ms, mfu 0.01%
step 6250: train loss 1.9509, val loss 2.0609
iter 6300: loss 1.2587, time 70.93ms, mfu 0.01%
iter 6400: loss 1.3309, time 115.63ms, mfu 0.01%
step 6500: train loss 1.9183, val loss 2.0399
saving checkpoint to out/proposed_margin_by_weight_alpha_2.0
iter 6500: loss 1.2317, time 1529.20ms, mfu 0.01%
iter 6600: loss 1.2122, time 120.34ms, mfu 0.01%
iter 6700: loss 1.2193, time 119.76ms, mfu 0.01%
step 6750: train loss 1.9357, val loss 2.0451
iter 6800: loss 1.2943, time 126.84ms, mfu 0.01%
iter 6900: loss 1.3816, time 86.20ms, mfu 0.01%
step 7000: train loss 1.9187, val loss 2.0471
iter 7000: loss 1.2781, time 1399.22ms, mfu 0.01%
iter 7100: loss 1.3630, time 131.22ms, mfu 0.01%
iter 7200: loss 1.2612, time 85.33ms, mfu 0.01%
step 7250: train loss 1.9293, val loss 2.0412
iter 7300: loss 1.2287, time 89.02ms, mfu 0.01%
iter 7400: loss 1.2212, 

iter 5700: loss 1.1580, time 81.68ms, mfu 0.01%
step 5750: train loss 2.7112, val loss 2.7860
saving checkpoint to out/proposed_margin_by_weight_alpha_1.0
iter 5800: loss 1.2983, time 132.83ms, mfu 0.01%
iter 5900: loss 1.2423, time 114.30ms, mfu 0.01%
step 6000: train loss 2.7246, val loss 2.7664
saving checkpoint to out/proposed_margin_by_weight_alpha_1.0
iter 6000: loss 1.2163, time 1649.95ms, mfu 0.01%
iter 6100: loss 1.2694, time 81.19ms, mfu 0.01%
iter 6200: loss 1.1145, time 128.15ms, mfu 0.01%
step 6250: train loss 2.7115, val loss 2.7786
iter 6300: loss 1.1369, time 82.07ms, mfu 0.01%
iter 6400: loss 1.2324, time 117.68ms, mfu 0.01%
step 6500: train loss 2.6923, val loss 2.7675
iter 6500: loss 1.1493, time 1553.85ms, mfu 0.01%
iter 6600: loss 1.1074, time 86.57ms, mfu 0.01%
iter 6700: loss 1.1369, time 95.23ms, mfu 0.01%
step 6750: train loss 2.7078, val loss 2.7758
iter 6800: loss 1.2099, time 104.58ms, mfu 0.01%
iter 6900: loss 1.2719, time 88.05ms, mfu 0.01%
step 7000: trai

iter 5300: loss 1.1899, time 122.23ms, mfu 0.01%
iter 5400: loss 1.1261, time 132.51ms, mfu 0.01%
step 5500: train loss 3.4951, val loss 3.5275
saving checkpoint to out/proposed_margin_by_weight_alpha_0.5
iter 5500: loss 1.1314, time 1423.34ms, mfu 0.01%
iter 5600: loss 1.1522, time 87.91ms, mfu 0.01%
iter 5700: loss 1.1560, time 83.18ms, mfu 0.01%
step 5750: train loss 3.4839, val loss 3.5279
iter 5800: loss 1.2380, time 96.05ms, mfu 0.01%
iter 5900: loss 1.1803, time 135.72ms, mfu 0.01%
step 6000: train loss 3.4929, val loss 3.5138
saving checkpoint to out/proposed_margin_by_weight_alpha_0.5
iter 6000: loss 1.1549, time 1653.49ms, mfu 0.01%
iter 6100: loss 1.2211, time 122.92ms, mfu 0.01%
iter 6200: loss 1.0608, time 111.46ms, mfu 0.01%
step 6250: train loss 3.4777, val loss 3.5180
iter 6300: loss 1.1133, time 143.09ms, mfu 0.01%
iter 6400: loss 1.1645, time 79.62ms, mfu 0.01%
step 6500: train loss 3.4636, val loss 3.5142
iter 6500: loss 1.0896, time 1422.91ms, mfu 0.01%
iter 6600: l

In [3]:
margin = 1.0
temperature = 0.44
args = {
    "out_dir": f"out/proposed_margin_{margin}_T_{temperature}_separated_embeddings",
    "model_type": "proposed",
    "margin": margin,
    "temperature": temperature,
    "separated_embeddings": "True"
}
run_training(args)
print(f'done for margin={margin}, T={temperature}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

step 7500: train loss 1.4546, val loss 1.6745
saving checkpoint to out/proposed_margin_1.0_T_0.44_separated_embeddings
iter 7500: loss 1.2560, time 3502.12ms, mfu 0.01%
iter 7600: loss 1.2327, time 627.48ms, mfu 0.01%
iter 7700: loss 1.1853, time 138.19ms, mfu 0.01%
step 7750: train loss 1.5003, val loss 1.6872
iter 7800: loss 1.3103, time 112.11ms, mfu 0.01%
iter 7900: loss 1.2317, time 177.54ms, mfu 0.01%
step 8000: train loss 1.4703, val loss 1.6856
iter 8000: loss 1.2805, time 2647.52ms, mfu 0.01%
done for margin=1.0, T=0.44


In [4]:
# workaround to save embeddings in old format

import torch

temperature = 0.44
for margin in [1.0]:
    # Path to your original checkpoint
    ckpt_path = f"out/proposed_margin_{margin}_T_{temperature}_separated_embeddings/ckpt.pt"
    # Path to save the fixed checkpoint

    # Load the checkpoint
    checkpoint = torch.load(ckpt_path, map_location='cpu')

    # Assuming the structure of the list of weights
    # is in transformer.wte.weights.X and lm_head.weights.X
    # Stack the list of weights for transformer.wte
    wte_weight_list = []
    for i in range(130):  # Assuming 130 vectors
        wte_weight_list.append(checkpoint['model']['transformer.wte.weights.{}'.format(i)])
    wte_stacked = torch.stack(wte_weight_list)  # Shape: (130, 128)

    # Stack the list of weights for lm_head
    lm_head_weight_list = []
    for i in range(130):  # Assuming 130 vectors
        lm_head_weight_list.append(checkpoint['model']['lm_head.weights.{}'.format(i)])
    lm_head_stacked = torch.stack(lm_head_weight_list)  # Shape: (130, 128)

    # Now replace the old list of weights with the stacked versions
    checkpoint['model']['transformer.wte.weight'] = wte_stacked
    checkpoint['model']['lm_head.weight'] = lm_head_stacked

    # Optionally, delete the old entries to avoid confusion
    for i in range(130):
        del checkpoint['model']['transformer.wte.weights.{}'.format(i)]
        del checkpoint['model']['lm_head.weights.{}'.format(i)]

    # Save the updated checkpoint
    fixed_ckpt_path = f"out/proposed_margin_{margin}_T_{temperature}_separated_embeddings/ckpt_fixed.pt"
    torch.save(checkpoint, fixed_ckpt_path)

    print(f"Fixed checkpoint saved to {fixed_ckpt_path}")


Fixed checkpoint saved to out/proposed_margin_1.0_T_0.44_separated_embeddings/ckpt_fixed.pt


/tmp/ipykernel_28/1374527990.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location='cpu')


In [5]:
!mv out/proposed_margin_1.0_T_0.44_separated_embeddings/ckpt.pt out/proposed_margin_1.0_T_0.44_separated_embeddings/ckpt_original.pt
!mv out/proposed_margin_1.0_T_0.44_separated_embeddings/ckpt_fixed.pt out/proposed_margin_1.0_T_0.44_separated_embeddings/ckpt.pt

In [5]:
for margin_by_weight_alpha in [2.0, 1.0, 0.5, 0.25]:
    args = {
        "out_dir": f"out/proposed_margin_by_weight_type_3_alpha_{margin_by_weight_alpha}",
        "model_type": "proposed",
        "margin_by_weight": "3",
        "margin_by_weight_alpha": margin_by_weight_alpha
    }
    run_training(args)
    print(f'done for proposed_margin_by_weight_type_3_alpha={margin_by_weight_alpha}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

iter 7600: loss 1.4446, time 84.47ms, mfu 0.01%
iter 7700: loss 1.5329, time 55.50ms, mfu 0.01%
step 7750: train loss 1.4369, val loss 1.6754
iter 7800: loss 1.4041, time 65.15ms, mfu 0.01%
iter 7900: loss 1.3969, time 79.27ms, mfu 0.01%
step 8000: train loss 1.4624, val loss 1.6347
iter 8000: loss 1.3548, time 1891.46ms, mfu 0.01%
done for proposed_margin_by_weight_type_3_alpha=2.0
Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
b

iter 7300: loss 1.3929, time 690.04ms, mfu 0.01%
iter 7400: loss 1.3515, time 43.29ms, mfu 0.01%
step 7500: train loss 1.4380, val loss 1.6134
saving checkpoint to out/proposed_margin_by_weight_type_3_alpha_1.0
iter 7500: loss 1.4704, time 1301.43ms, mfu 0.01%
iter 7600: loss 1.4412, time 251.26ms, mfu 0.01%
iter 7700: loss 1.5256, time 70.84ms, mfu 0.01%
step 7750: train loss 1.4389, val loss 1.6731
iter 7800: loss 1.3794, time 67.63ms, mfu 0.01%
iter 7900: loss 1.4425, time 60.84ms, mfu 0.01%
step 8000: train loss 1.4541, val loss 1.6297
iter 8000: loss 1.3412, time 795.52ms, mfu 0.01%
done for proposed_margin_by_weight_type_3_alpha=1.0
Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this s

iter 7100: loss 1.5333, time 65.94ms, mfu 0.02%
iter 7200: loss 1.4371, time 68.19ms, mfu 0.02%
step 7250: train loss 1.4330, val loss 1.6269
iter 7300: loss 1.3741, time 162.83ms, mfu 0.01%
iter 7400: loss 1.3303, time 322.65ms, mfu 0.01%
step 7500: train loss 1.4305, val loss 1.5990
saving checkpoint to out/proposed_margin_by_weight_type_3_alpha_0.5
iter 7500: loss 1.4361, time 1139.56ms, mfu 0.01%
iter 7600: loss 1.4294, time 403.35ms, mfu 0.01%
iter 7700: loss 1.5109, time 48.83ms, mfu 0.01%
step 7750: train loss 1.4220, val loss 1.6532
iter 7800: loss 1.3736, time 114.25ms, mfu 0.01%
iter 7900: loss 1.3766, time 674.36ms, mfu 0.01%
step 8000: train loss 1.4338, val loss 1.6134
iter 8000: loss 1.3222, time 2464.55ms, mfu 0.01%
done for proposed_margin_by_weight_type_3_alpha=0.5
Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_int

saving checkpoint to out/proposed_margin_by_weight_type_3_alpha_0.25
iter 6500: loss 1.3153, time 882.91ms, mfu 0.01%
iter 6600: loss 1.2928, time 72.08ms, mfu 0.01%
iter 6700: loss 1.3153, time 42.18ms, mfu 0.01%
step 6750: train loss 1.5612, val loss 1.7063
iter 6800: loss 1.3784, time 46.42ms, mfu 0.02%
iter 6900: loss 1.4563, time 66.56ms, mfu 0.02%
step 7000: train loss 1.5364, val loss 1.7068
iter 7000: loss 1.3637, time 665.17ms, mfu 0.02%
iter 7100: loss 1.4462, time 46.71ms, mfu 0.02%
iter 7200: loss 1.3505, time 68.09ms, mfu 0.02%
step 7250: train loss 1.5551, val loss 1.6948
saving checkpoint to out/proposed_margin_by_weight_type_3_alpha_0.25
iter 7300: loss 1.3002, time 59.48ms, mfu 0.02%
iter 7400: loss 1.2641, time 92.64ms, mfu 0.02%
step 7500: train loss 1.5496, val loss 1.6871
saving checkpoint to out/proposed_margin_by_weight_type_3_alpha_0.25
iter 7500: loss 1.3763, time 786.28ms, mfu 0.02%
iter 7600: loss 1.3398, time 51.91ms, mfu 0.02%
iter 7700: loss 1.4218, time 4

In [19]:
for margin_by_weight_alpha in [0.125, 0.0625, 0.04, 0.02, 0.01]:
    args = {
        "out_dir": f"out/proposed_margin_by_weight_type_3_alpha_{margin_by_weight_alpha}",
        "model_type": "proposed",
        "margin_by_weight": "3",
        "margin_by_weight_alpha": margin_by_weight_alpha
    }
    run_training(args)
    print(f'done for proposed_margin_by_weight_type_3_alpha={margin_by_weight_alpha}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

iter 7200: loss 1.2143, time 57.72ms, mfu 0.01%
step 7250: train loss 2.4267, val loss 2.5098
saving checkpoint to out/proposed_margin_by_weight_type_3_alpha_0.125
iter 7300: loss 1.1712, time 306.43ms, mfu 0.01%
iter 7400: loss 1.1366, time 70.51ms, mfu 0.01%
step 7500: train loss 2.4230, val loss 2.5058
saving checkpoint to out/proposed_margin_by_weight_type_3_alpha_0.125
iter 7500: loss 1.2368, time 962.28ms, mfu 0.01%
iter 7600: loss 1.1871, time 52.80ms, mfu 0.01%
iter 7700: loss 1.2545, time 87.30ms, mfu 0.01%
step 7750: train loss 2.4317, val loss 2.5321
iter 7800: loss 1.1486, time 41.98ms, mfu 0.02%
iter 7900: loss 1.1915, time 83.11ms, mfu 0.02%
step 8000: train loss 2.4314, val loss 2.5104
iter 8000: loss 1.1416, time 926.91ms, mfu 0.01%
done for proposed_margin_by_weight_type_3_alpha=0.125
Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakes

saving checkpoint to out/proposed_margin_by_weight_type_3_alpha_0.0625
iter 6800: loss 1.1363, time 322.18ms, mfu 0.01%
iter 6900: loss 1.2149, time 75.26ms, mfu 0.01%
step 7000: train loss 3.4407, val loss 3.5035
saving checkpoint to out/proposed_margin_by_weight_type_3_alpha_0.0625
iter 7000: loss 1.1370, time 974.25ms, mfu 0.01%
iter 7100: loss 1.2080, time 54.64ms, mfu 0.01%
iter 7200: loss 1.1006, time 109.92ms, mfu 0.01%
step 7250: train loss 3.4688, val loss 3.5150
iter 7300: loss 1.0981, time 48.37ms, mfu 0.01%
iter 7400: loss 1.0709, time 112.69ms, mfu 0.01%
step 7500: train loss 3.4653, val loss 3.5088
iter 7500: loss 1.1568, time 1355.23ms, mfu 0.01%
iter 7600: loss 1.1063, time 47.41ms, mfu 0.01%
iter 7700: loss 1.1636, time 45.97ms, mfu 0.02%
step 7750: train loss 3.4646, val loss 3.5221
iter 7800: loss 1.0677, time 116.66ms, mfu 0.02%
iter 7900: loss 1.1014, time 46.23ms, mfu 0.02%
step 8000: train loss 3.4565, val loss 3.4952
saving checkpoint to out/proposed_margin_by_w

iter 7400: loss 1.0080, time 83.77ms, mfu 0.02%
step 7500: train loss 4.3314, val loss 4.3388
iter 7500: loss 1.1231, time 716.09ms, mfu 0.01%
iter 7600: loss 1.0545, time 77.41ms, mfu 0.01%
iter 7700: loss 1.1282, time 63.15ms, mfu 0.02%
step 7750: train loss 4.3090, val loss 4.3332
iter 7800: loss 1.0165, time 44.13ms, mfu 0.02%
iter 7900: loss 1.0453, time 61.42ms, mfu 0.02%
step 8000: train loss 4.3132, val loss 4.3188
iter 8000: loss 0.9926, time 1113.71ms, mfu 0.02%
done for proposed_margin_by_weight_type_3_alpha=0.04
Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command


wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

warmup_iters = 100 # not super necessary potentially

# on macbook also add
# device = 'cpu'  # run on cpu only
# compile = False # do not torch compile the model

Overriding: out_dir = out/proposed_margin_by_weight_type_3_alpha_0.01
Overriding: model_type = proposed
Overriding: margin_by_weight = 3
Overriding: margin_by_weight_alpha = 0.01
tokens per iteration will be: 768
found vocab_size = 65 (insid

In [8]:
for margin_by_weight_alpha in [0.5, 0.25, 0.125, 0.0625]:
    args = {
        "out_dir": f"out/proposed_margin_by_weight_type_3_alpha_{margin_by_weight_alpha}_SE",
        "model_type": "proposed",
        "margin_by_weight": "3",
        "margin_by_weight_alpha": margin_by_weight_alpha,
        "separated_embeddings": "True"
    }
    run_training(args)
    print(f'done for proposed_margin_by_weight_type_3_alpha={margin_by_weight_alpha}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

iter 7800: loss 1.4859, time 72.67ms, mfu 0.01%
iter 7900: loss 1.4124, time 97.95ms, mfu 0.01%
step 8000: train loss 1.3864, val loss 1.6266
iter 8000: loss 1.4427, time 1208.92ms, mfu 0.01%
done for proposed_margin_by_weight_type_3_alpha=0.5
Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.

step 7000: train loss 1.5455, val loss 1.7288
iter 7000: loss 1.4590, time 1030.17ms, mfu 0.01%
iter 7100: loss 1.3706, time 105.37ms, mfu 0.01%
iter 7200: loss 1.2911, time 79.98ms, mfu 0.01%
step 7250: train loss 1.5433, val loss 1.7216
iter 7300: loss 1.4014, time 78.62ms, mfu 0.01%
iter 7400: loss 1.3871, time 90.44ms, mfu 0.01%
step 7500: train loss 1.5199, val loss 1.7072
saving checkpoint to out/proposed_margin_by_weight_type_3_alpha_0.25_SE
iter 7500: loss 1.3460, time 1236.87ms, mfu 0.01%
iter 7600: loss 1.3124, time 109.67ms, mfu 0.01%
iter 7700: loss 1.2751, time 120.15ms, mfu 0.01%
step 7750: train loss 1.5481, val loss 1.7143
iter 7800: loss 1.3996, time 113.01ms, mfu 0.01%
iter 7900: loss 1.3156, time 126.94ms, mfu 0.01%
step 8000: train loss 1.5268, val loss 1.7158
iter 8000: loss 1.3952, time 1488.81ms, mfu 0.01%
done for proposed_margin_by_weight_type_3_alpha=0.25
Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare m

step 6000: train loss 2.4098, val loss 2.5079
iter 6000: loss 1.2615, time 1113.99ms, mfu 0.01%
iter 6100: loss 1.2855, time 116.18ms, mfu 0.01%
iter 6200: loss 1.1752, time 99.80ms, mfu 0.01%
step 6250: train loss 2.4327, val loss 2.5011
iter 6300: loss 1.2884, time 114.46ms, mfu 0.01%
iter 6400: loss 1.1230, time 83.37ms, mfu 0.01%
step 6500: train loss 2.4204, val loss 2.5035
iter 6500: loss 1.2217, time 1212.44ms, mfu 0.01%
iter 6600: loss 1.2868, time 128.14ms, mfu 0.01%
iter 6700: loss 1.2118, time 108.29ms, mfu 0.01%
step 6750: train loss 2.4096, val loss 2.4960
saving checkpoint to out/proposed_margin_by_weight_type_3_alpha_0.125_SE
iter 6800: loss 1.2324, time 115.49ms, mfu 0.01%
iter 6900: loss 1.2133, time 118.54ms, mfu 0.01%
step 7000: train loss 2.3942, val loss 2.4998
iter 7000: loss 1.3134, time 1241.54ms, mfu 0.01%
iter 7100: loss 1.2442, time 83.23ms, mfu 0.01%
iter 7200: loss 1.1361, time 108.87ms, mfu 0.01%
step 7250: train loss 2.3982, val loss 2.4966
iter 7300: los

iter 5400: loss 1.1868, time 237.75ms, mfu 0.01%
step 5500: train loss 3.4845, val loss 3.5117
saving checkpoint to out/proposed_margin_by_weight_type_3_alpha_0.0625_SE
iter 5500: loss 1.2106, time 1642.52ms, mfu 0.01%
iter 5600: loss 1.1611, time 142.48ms, mfu 0.01%
iter 5700: loss 1.0938, time 136.15ms, mfu 0.01%
step 5750: train loss 3.4710, val loss 3.5073
saving checkpoint to out/proposed_margin_by_weight_type_3_alpha_0.0625_SE
iter 5800: loss 1.1334, time 124.79ms, mfu 0.01%
iter 5900: loss 1.1062, time 125.47ms, mfu 0.01%
step 6000: train loss 3.4587, val loss 3.5138
iter 6000: loss 1.1016, time 1260.50ms, mfu 0.01%
iter 6100: loss 1.1503, time 122.97ms, mfu 0.01%
iter 6200: loss 1.1246, time 110.91ms, mfu 0.01%
step 6250: train loss 3.4586, val loss 3.4937
saving checkpoint to out/proposed_margin_by_weight_type_3_alpha_0.0625_SE
iter 6300: loss 1.1189, time 106.22ms, mfu 0.01%
iter 6400: loss 1.0513, time 191.62ms, mfu 0.01%
step 6500: train loss 3.4839, val loss 3.5191
iter 65

In [20]:
for margin_by_weight_alpha in [0.04, 0.02, 0.01]:
    args = {
        "out_dir": f"out/proposed_margin_by_weight_type_3_alpha_{margin_by_weight_alpha}_SE",
        "model_type": "proposed",
        "margin_by_weight": "3",
        "margin_by_weight_alpha": margin_by_weight_alpha,
        "separated_embeddings": "True"
    }
    run_training(args)
    print(f'done for proposed_margin_by_weight_type_3_alpha={margin_by_weight_alpha}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

warmup_iters = 100 # not super necessary potentially

# on macbook also add
# device = 'cpu'  # run on cpu only
# compile = False # do not torch compile the model

Overriding: out_dir = out/proposed_margin_by_weight_type_3_alpha_0.02_SE
Overriding: model_type = proposed
Overriding: margin_by_weight = 3
Overriding: margin_by_weight_alpha = 0.02
Overriding: separated_embedd

# compile = False # do not torch compile the model

Overriding: out_dir = out/proposed_margin_by_weight_type_3_alpha_0.01_SE
Overriding: model_type = proposed
Overriding: margin_by_weight = 3
Overriding: margin_by_weight_alpha = 0.01
Overriding: separated_embeddings = True
tokens per iteration will be: 768
found vocab_size = 65 (inside data/shakespeare_char/meta.pkl)
Initializing a new model from scratch
number of parameters: 0.80M
number of parameters: 0.80M
num decayed parameter tensors: 147, with 811,264 parameters
num non-decayed parameter tensors: 9, with 1,152 parameters
using fused AdamW: False
step 0: train loss 4.8737, val loss 4.8674
iter 0: loss 4.1240, time 1904.48ms, mfu -100.00%
iter 100: loss 1.8946, time 131.22ms, mfu 0.01%
iter 200: loss 1.7997, time 154.45ms, mfu 0.01%
step 250: train loss 4.6125, val loss 4.6144
saving checkpoint to out/proposed_margin_by_weight_type_3_alpha_0.01_SE
iter 300: loss 1.6885, time 98.54ms, mfu 0.01%
iter 400: loss 1.7841, time 143.45ms, 

In [21]:
# workaround to save embeddings in old format

import torch

for margin_by_weight_alpha in [0.04, 0.02, 0.01]: # [0.5, 0.25, 0.125, 0.0625]:
    
    # Path to your original checkpoint
    out_path = f"out/proposed_margin_by_weight_type_3_alpha_{margin_by_weight_alpha}_SE"
    ckpt_path = f"{out_path}/ckpt.pt"
    # Path to save the fixed checkpoint

    # Load the checkpoint
    checkpoint = torch.load(ckpt_path, map_location='cpu')
    old_ckpt_path = f"{out_path}/ckpt_old.pt"
    torch.save(checkpoint, old_ckpt_path)

    # Assuming the structure of the list of weights
    # is in transformer.wte.weights.X and lm_head.weights.X
    # Stack the list of weights for transformer.wte
    wte_weight_list = []
    for i in range(130):  # Assuming 130 vectors
        wte_weight_list.append(checkpoint['model']['transformer.wte.weights.{}'.format(i)])
    wte_stacked = torch.stack(wte_weight_list)  # Shape: (130, 128)

    # Stack the list of weights for lm_head
    lm_head_weight_list = []
    for i in range(130):  # Assuming 130 vectors
        lm_head_weight_list.append(checkpoint['model']['lm_head.weights.{}'.format(i)])
    lm_head_stacked = torch.stack(lm_head_weight_list)  # Shape: (130, 128)

    # Now replace the old list of weights with the stacked versions
    checkpoint['model']['transformer.wte.weight'] = wte_stacked
    checkpoint['model']['lm_head.weight'] = lm_head_stacked

    # Optionally, delete the old entries to avoid confusion
    for i in range(130):
        del checkpoint['model']['transformer.wte.weights.{}'.format(i)]
        del checkpoint['model']['lm_head.weights.{}'.format(i)]

    # Save the updated checkpoint
    fixed_ckpt_path = f"{out_path}/ckpt.pt"
    torch.save(checkpoint, fixed_ckpt_path)

    print(f"Fixed checkpoint saved to {fixed_ckpt_path}")


/tmp/ipykernel_341/1649692800.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location='cpu')


Fixed checkpoint saved to out/proposed_margin_by_weight_type_3_alpha_0.04_SE/ckpt.pt
Fixed checkpoint saved to out/proposed_margin_by_weight_type_3_alpha_0.02_SE/ckpt.pt
Fixed checkpoint saved to out/proposed_margin_by_weight_type_3_alpha_0.01_SE/ckpt.pt


In [9]:
margin = 1.0
for temperature in [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3]:
    args = {
        "out_dir": f"out/proposed_margin_{margin}_T_{temperature}_separated_embeddings",
        "model_type": "proposed",
        "margin": margin,
        "temperature": temperature,
        "separated_embeddings": "True"
    }
    run_training(args)
    print(f'done for margin={margin}, T={temperature}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

iter 5300: loss 1.1388, time 102.30ms, mfu 0.01%
iter 5400: loss 1.3078, time 109.52ms, mfu 0.01%
step 5500: train loss 2.0536, val loss 2.1218
saving checkpoint to out/proposed_margin_1.0_T_0.9_separated_embeddings
iter 5500: loss 1.3391, time 1432.67ms, mfu 0.01%
iter 5600: loss 1.3224, time 114.13ms, mfu 0.01%
iter 5700: loss 1.2293, time 92.40ms, mfu 0.01%
step 5750: train loss 2.0110, val loss 2.1058
saving checkpoint to out/proposed_margin_1.0_T_0.9_separated_embeddings
iter 5800: loss 1.2462, time 98.84ms, mfu 0.01%
iter 5900: loss 1.2553, time 114.23ms, mfu 0.01%
step 6000: train loss 2.0085, val loss 2.1310
iter 6000: loss 1.3036, time 1123.30ms, mfu 0.01%
iter 6100: loss 1.3296, time 84.58ms, mfu 0.01%
iter 6200: loss 1.2205, time 96.53ms, mfu 0.01%
step 6250: train loss 2.0274, val loss 2.1129
iter 6300: loss 1.3308, time 88.21ms, mfu 0.01%
iter 6400: loss 1.1606, time 125.29ms, mfu 0.01%
step 6500: train loss 2.0137, val loss 2.1190
iter 6500: loss 1.2355, time 1118.82ms, m

iter 5000: loss 1.4123, time 1414.98ms, mfu 0.01%
iter 5100: loss 1.2972, time 93.74ms, mfu 0.01%
iter 5200: loss 1.2907, time 86.54ms, mfu 0.01%
step 5250: train loss 1.8955, val loss 2.0007
saving checkpoint to out/proposed_margin_1.0_T_0.8_separated_embeddings
iter 5300: loss 1.1390, time 133.15ms, mfu 0.01%
iter 5400: loss 1.3212, time 89.13ms, mfu 0.01%
step 5500: train loss 1.9217, val loss 1.9970
saving checkpoint to out/proposed_margin_1.0_T_0.8_separated_embeddings
iter 5500: loss 1.3506, time 1119.03ms, mfu 0.01%
iter 5600: loss 1.3152, time 81.73ms, mfu 0.01%
iter 5700: loss 1.2148, time 134.15ms, mfu 0.01%
step 5750: train loss 1.8621, val loss 1.9752
saving checkpoint to out/proposed_margin_1.0_T_0.8_separated_embeddings
iter 5800: loss 1.2424, time 96.17ms, mfu 0.01%
iter 5900: loss 1.2645, time 256.73ms, mfu 0.01%
step 6000: train loss 1.8582, val loss 1.9935
iter 6000: loss 1.2927, time 1736.86ms, mfu 0.01%
iter 6100: loss 1.3495, time 82.19ms, mfu 0.01%
iter 6200: loss

iter 4500: loss 1.3647, time 1386.83ms, mfu 0.01%
iter 4600: loss 1.4030, time 115.19ms, mfu 0.01%
iter 4700: loss 1.2884, time 133.06ms, mfu 0.01%
step 4750: train loss 1.7836, val loss 1.9283
saving checkpoint to out/proposed_margin_1.0_T_0.7_separated_embeddings
iter 4800: loss 1.3409, time 89.87ms, mfu 0.01%
iter 4900: loss 1.2090, time 116.72ms, mfu 0.01%
step 5000: train loss 1.7349, val loss 1.8961
saving checkpoint to out/proposed_margin_1.0_T_0.7_separated_embeddings
iter 5000: loss 1.4065, time 1206.69ms, mfu 0.01%
iter 5100: loss 1.3015, time 101.36ms, mfu 0.01%
iter 5200: loss 1.2752, time 84.88ms, mfu 0.01%
step 5250: train loss 1.7544, val loss 1.8696
saving checkpoint to out/proposed_margin_1.0_T_0.7_separated_embeddings
iter 5300: loss 1.1399, time 110.98ms, mfu 0.01%
iter 5400: loss 1.3057, time 152.84ms, mfu 0.01%
step 5500: train loss 1.7818, val loss 1.8663
saving checkpoint to out/proposed_margin_1.0_T_0.7_separated_embeddings
iter 5500: loss 1.3619, time 1494.23ms

iter 4100: loss 1.2499, time 115.60ms, mfu 0.01%
iter 4200: loss 1.2332, time 85.45ms, mfu 0.01%
step 4250: train loss 1.6877, val loss 1.8447
saving checkpoint to out/proposed_margin_1.0_T_0.6_separated_embeddings
iter 4300: loss 1.2692, time 171.78ms, mfu 0.01%
iter 4400: loss 1.4446, time 109.08ms, mfu 0.01%
step 4500: train loss 1.6862, val loss 1.8492
iter 4500: loss 1.3722, time 1447.03ms, mfu 0.01%
iter 4600: loss 1.3996, time 127.67ms, mfu 0.01%
iter 4700: loss 1.2977, time 95.11ms, mfu 0.01%
step 4750: train loss 1.6798, val loss 1.8421
saving checkpoint to out/proposed_margin_1.0_T_0.6_separated_embeddings
iter 4800: loss 1.3275, time 88.02ms, mfu 0.01%
iter 4900: loss 1.2018, time 104.07ms, mfu 0.01%
step 5000: train loss 1.6249, val loss 1.8117
saving checkpoint to out/proposed_margin_1.0_T_0.6_separated_embeddings
iter 5000: loss 1.4299, time 1707.25ms, mfu 0.01%
iter 5100: loss 1.3275, time 87.61ms, mfu 0.01%
iter 5200: loss 1.3036, time 92.67ms, mfu 0.01%
step 5250: trai

step 3500: train loss 1.6732, val loss 1.8337
iter 3500: loss 1.3107, time 1452.39ms, mfu 0.01%
iter 3600: loss 1.3451, time 117.25ms, mfu 0.01%
iter 3700: loss 1.3239, time 78.10ms, mfu 0.01%
step 3750: train loss 1.6570, val loss 1.7983
saving checkpoint to out/proposed_margin_1.0_T_0.5_separated_embeddings
iter 3800: loss 1.3371, time 110.15ms, mfu 0.01%
iter 3900: loss 1.2806, time 118.89ms, mfu 0.01%
step 4000: train loss 1.6191, val loss 1.8200
iter 4000: loss 1.4426, time 1428.38ms, mfu 0.01%
iter 4100: loss 1.2836, time 137.32ms, mfu 0.01%
iter 4200: loss 1.2392, time 113.11ms, mfu 0.01%
step 4250: train loss 1.6200, val loss 1.7848
saving checkpoint to out/proposed_margin_1.0_T_0.5_separated_embeddings
iter 4300: loss 1.2594, time 108.56ms, mfu 0.01%
iter 4400: loss 1.4336, time 133.30ms, mfu 0.01%
step 4500: train loss 1.6210, val loss 1.7980
iter 4500: loss 1.3850, time 1218.25ms, mfu 0.01%
iter 4600: loss 1.3971, time 135.61ms, mfu 0.01%
iter 4700: loss 1.3161, time 103.24m

iter 2900: loss 1.4201, time 146.77ms, mfu 0.01%
step 3000: train loss 1.8146, val loss 1.9514
iter 3000: loss 1.4521, time 1048.42ms, mfu 0.01%
iter 3100: loss 1.3730, time 260.22ms, mfu 0.01%
iter 3200: loss 1.4208, time 114.06ms, mfu 0.01%
step 3250: train loss 1.7397, val loss 1.8733
saving checkpoint to out/proposed_margin_1.0_T_0.4_separated_embeddings
iter 3300: loss 1.4810, time 79.96ms, mfu 0.01%
iter 3400: loss 1.4387, time 302.94ms, mfu 0.01%
step 3500: train loss 1.7194, val loss 1.9108
iter 3500: loss 1.3194, time 1124.41ms, mfu 0.01%
iter 3600: loss 1.3516, time 144.64ms, mfu 0.01%
iter 3700: loss 1.3269, time 76.22ms, mfu 0.01%
step 3750: train loss 1.6783, val loss 1.8315
saving checkpoint to out/proposed_margin_1.0_T_0.4_separated_embeddings
iter 3800: loss 1.3651, time 121.49ms, mfu 0.01%
iter 3900: loss 1.2769, time 71.60ms, mfu 0.01%
step 4000: train loss 1.6539, val loss 1.8856
iter 4000: loss 1.4509, time 1186.20ms, mfu 0.01%
iter 4100: loss 1.2934, time 111.79ms,

iter 2700: loss 1.4514, time 108.52ms, mfu 0.01%
step 2750: train loss 2.0312, val loss 2.1636
saving checkpoint to out/proposed_margin_1.0_T_0.3_separated_embeddings
iter 2800: loss 1.6342, time 110.18ms, mfu 0.01%
iter 2900: loss 1.4258, time 89.16ms, mfu 0.01%
step 3000: train loss 2.0237, val loss 2.2043
iter 3000: loss 1.4526, time 1225.82ms, mfu 0.01%
iter 3100: loss 1.3754, time 119.08ms, mfu 0.01%
iter 3200: loss 1.4061, time 131.85ms, mfu 0.01%
step 3250: train loss 1.9328, val loss 2.1000
saving checkpoint to out/proposed_margin_1.0_T_0.3_separated_embeddings
iter 3300: loss 1.4888, time 119.90ms, mfu 0.01%
iter 3400: loss 1.4583, time 78.49ms, mfu 0.01%
step 3500: train loss 1.9062, val loss 2.1479
iter 3500: loss 1.3320, time 1062.69ms, mfu 0.01%
iter 3600: loss 1.3494, time 99.71ms, mfu 0.01%
iter 3700: loss 1.3502, time 133.08ms, mfu 0.01%
step 3750: train loss 1.8616, val loss 2.0802
saving checkpoint to out/proposed_margin_1.0_T_0.3_separated_embeddings
iter 3800: loss 

In [24]:
margin = 1.0
for temperature in [1.0]:
    args = {
        "out_dir": f"out/proposed_margin_{margin}_T_{temperature}_separated_embeddings",
        "model_type": "proposed",
        "margin": margin,
        "temperature": temperature,
        "separated_embeddings": "True"
    }
    run_training(args)
    print(f'done for margin={margin}, T={temperature}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

step 7500: train loss 2.1152, val loss 2.2345
saving checkpoint to out/proposed_margin_1.0_T_1.0_separated_embeddings
iter 7500: loss 1.2565, time 1637.51ms, mfu 0.01%
iter 7600: loss 1.2141, time 134.54ms, mfu 0.01%
iter 7700: loss 1.1772, time 87.71ms, mfu 0.01%
step 7750: train loss 2.1249, val loss 2.2322
saving checkpoint to out/proposed_margin_1.0_T_1.0_separated_embeddings
iter 7800: loss 1.2702, time 63.71ms, mfu 0.01%
iter 7900: loss 1.2139, time 80.01ms, mfu 0.01%
step 8000: train loss 2.1069, val loss 2.2356
iter 8000: loss 1.2710, time 983.52ms, mfu 0.01%
done for margin=1.0, T=1.0


In [25]:
# workaround to save embeddings in old format

import torch
margin = 1.0
for temperature in [1.0]: # , 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3]:
    out_path = f"out/proposed_margin_{margin}_T_{temperature}_separated_embeddings"
    ckpt_path = f"{out_path}/ckpt.pt"
    # Path to save the fixed checkpoint

    # Load the checkpoint
    checkpoint = torch.load(ckpt_path, map_location='cpu')
    old_ckpt_path = f"{out_path}/ckpt_old.pt"
    torch.save(checkpoint, old_ckpt_path)

    # Assuming the structure of the list of weights
    # is in transformer.wte.weights.X and lm_head.weights.X
    # Stack the list of weights for transformer.wte
    wte_weight_list = []
    for i in range(130):  # Assuming 130 vectors
        wte_weight_list.append(checkpoint['model']['transformer.wte.weights.{}'.format(i)])
    wte_stacked = torch.stack(wte_weight_list)  # Shape: (130, 128)

    # Stack the list of weights for lm_head
    lm_head_weight_list = []
    for i in range(130):  # Assuming 130 vectors
        lm_head_weight_list.append(checkpoint['model']['lm_head.weights.{}'.format(i)])
    lm_head_stacked = torch.stack(lm_head_weight_list)  # Shape: (130, 128)

    # Now replace the old list of weights with the stacked versions
    checkpoint['model']['transformer.wte.weight'] = wte_stacked
    checkpoint['model']['lm_head.weight'] = lm_head_stacked

    # Optionally, delete the old entries to avoid confusion
    for i in range(130):
        del checkpoint['model']['transformer.wte.weights.{}'.format(i)]
        del checkpoint['model']['lm_head.weights.{}'.format(i)]

    # Save the updated checkpoint
    fixed_ckpt_path = f"{out_path}/ckpt.pt"
    torch.save(checkpoint, fixed_ckpt_path)

    print(f"Fixed checkpoint saved to {fixed_ckpt_path}")


/tmp/ipykernel_341/2964746794.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location='cpu')


Fixed checkpoint saved to out/proposed_margin_1.0_T_1.0_separated_embeddings/ckpt.pt


In [29]:
for margin in [4.0]: # [1.0, 2.0]:
    args = {
        "out_dir": f"out/proposed_margin_{margin}_wo_weight_tying_SE",
        "model_type": "proposed",
        "margin": margin,
        "weight_tying": "False",
        "separated_embeddings": "True"
    }
    run_training(args)
    print(f'done for margin={margin}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

step 8000: train loss 1.4052, val loss 1.6223
iter 8000: loss 1.4203, time 1267.38ms, mfu 0.01%
done for margin=4.0


In [26]:
#baseline wo weight_tying
args = {
    "out_dir": "out/baseline_wo_weight_tying_SE",
    "weight_tying": "False",
    "separated_embeddings": "True",
    "model_type": "baseline"
}
run_training(args)

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

In [30]:
# workaround to save embeddings in old format

import torch
margin = 1.0
for model_name in [
    'proposed_margin_1.0_wo_weight_tying_SE',
    'proposed_margin_2.0_wo_weight_tying_SE',
    'proposed_margin_4.0_wo_weight_tying_SE'
    "baseline_wo_weight_tying_SE",
]:
    out_path = f"out/{model_name}"
    ckpt_path = f"{out_path}/ckpt.pt"
    # Path to save the fixed checkpoint

    # Load the checkpoint
    checkpoint = torch.load(ckpt_path, map_location='cpu')
    old_ckpt_path = f"{out_path}/ckpt_old.pt"
    torch.save(checkpoint, old_ckpt_path)

    # Assuming the structure of the list of weights
    # is in transformer.wte.weights.X and lm_head.weights.X
    # Stack the list of weights for transformer.wte
    wte_weight_list = []
    for i in range(130):  # Assuming 130 vectors
        wte_weight_list.append(checkpoint['model']['transformer.wte.weights.{}'.format(i)])
    wte_stacked = torch.stack(wte_weight_list)  # Shape: (130, 128)

    # Stack the list of weights for lm_head
    lm_head_weight_list = []
    for i in range(130):  # Assuming 130 vectors
        lm_head_weight_list.append(checkpoint['model']['lm_head.weights.{}'.format(i)])
    lm_head_stacked = torch.stack(lm_head_weight_list)  # Shape: (130, 128)

    # Now replace the old list of weights with the stacked versions
    checkpoint['model']['transformer.wte.weight'] = wte_stacked
    checkpoint['model']['lm_head.weight'] = lm_head_stacked

    # Optionally, delete the old entries to avoid confusion
    for i in range(130):
        del checkpoint['model']['transformer.wte.weights.{}'.format(i)]
        del checkpoint['model']['lm_head.weights.{}'.format(i)]

    # Save the updated checkpoint
    fixed_ckpt_path = f"{out_path}/ckpt.pt"
    torch.save(checkpoint, fixed_ckpt_path)

    print(f"Fixed checkpoint saved to {fixed_ckpt_path}")


/tmp/ipykernel_341/3673994607.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location='cpu')


Fixed checkpoint saved to out/proposed_margin_4.0_wo_weight_tying_SE/ckpt.pt


In [22]:
for margin_by_weight_alpha in [0.0625]:
    args = {
        "out_dir": f"out/proposed_margin_by_weight_type_3_alpha_{margin_by_weight_alpha}_SE_2",
        "model_type": "proposed",
        "margin_by_weight": "3",
        "margin_by_weight_alpha": margin_by_weight_alpha,
        "separated_embeddings": "True"
    }
    run_training(args)
    print(f'done for proposed_margin_by_weight_type_3_alpha={margin_by_weight_alpha}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

step 7750: train loss 3.4635, val loss 3.5110
saving checkpoint to out/proposed_margin_by_weight_type_3_alpha_0.0625_SE_2
iter 7800: loss 1.1612, time 92.44ms, mfu 0.01%
iter 7900: loss 1.0898, time 181.08ms, mfu 0.01%
step 8000: train loss 3.4513, val loss 3.5071
saving checkpoint to out/proposed_margin_by_weight_type_3_alpha_0.0625_SE_2
iter 8000: loss 1.1267, time 897.41ms, mfu 0.01%
done for proposed_margin_by_weight_type_3_alpha=0.0625


In [23]:
# workaround to save embeddings in old format

import torch
margin = 1.0
for model_name in [
    'proposed_margin_by_weight_type_3_alpha_0.0625_SE_2',
]:
    out_path = f"out/{model_name}"
    ckpt_path = f"{out_path}/ckpt.pt"
    # Path to save the fixed checkpoint

    # Load the checkpoint
    checkpoint = torch.load(ckpt_path, map_location='cpu')
    old_ckpt_path = f"{out_path}/ckpt_old.pt"
    torch.save(checkpoint, old_ckpt_path)

    # Assuming the structure of the list of weights
    # is in transformer.wte.weights.X and lm_head.weights.X
    # Stack the list of weights for transformer.wte
    wte_weight_list = []
    for i in range(130):  # Assuming 130 vectors
        wte_weight_list.append(checkpoint['model']['transformer.wte.weights.{}'.format(i)])
    wte_stacked = torch.stack(wte_weight_list)  # Shape: (130, 128)

    # Stack the list of weights for lm_head
    lm_head_weight_list = []
    for i in range(130):  # Assuming 130 vectors
        lm_head_weight_list.append(checkpoint['model']['lm_head.weights.{}'.format(i)])
    lm_head_stacked = torch.stack(lm_head_weight_list)  # Shape: (130, 128)

    # Now replace the old list of weights with the stacked versions
    checkpoint['model']['transformer.wte.weight'] = wte_stacked
    checkpoint['model']['lm_head.weight'] = lm_head_stacked

    # Optionally, delete the old entries to avoid confusion
    for i in range(130):
        del checkpoint['model']['transformer.wte.weights.{}'.format(i)]
        del checkpoint['model']['lm_head.weights.{}'.format(i)]

    # Save the updated checkpoint
    fixed_ckpt_path = f"{out_path}/ckpt.pt"
    torch.save(checkpoint, fixed_ckpt_path)

    print(f"Fixed checkpoint saved to {fixed_ckpt_path}")


/tmp/ipykernel_341/1850883804.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location='cpu')


Fixed checkpoint saved to out/proposed_margin_by_weight_type_3_alpha_0.0625_SE_2/ckpt.pt


In [31]:
def fix_ckpt(model_name):
    out_path = f"out/{model_name}"
    ckpt_path = f"{out_path}/ckpt.pt"
    # Path to save the fixed checkpoint

    # Load the checkpoint
    checkpoint = torch.load(ckpt_path, map_location='cpu')
    old_ckpt_path = f"{out_path}/ckpt_old.pt"
    torch.save(checkpoint, old_ckpt_path)

    # Assuming the structure of the list of weights
    # is in transformer.wte.weights.X and lm_head.weights.X
    # Stack the list of weights for transformer.wte
    wte_weight_list = []
    for i in range(130):  # Assuming 130 vectors
        wte_weight_list.append(checkpoint['model']['transformer.wte.weights.{}'.format(i)])
    wte_stacked = torch.stack(wte_weight_list)  # Shape: (130, 128)

    # Stack the list of weights for lm_head
    lm_head_weight_list = []
    for i in range(130):  # Assuming 130 vectors
        lm_head_weight_list.append(checkpoint['model']['lm_head.weights.{}'.format(i)])
    lm_head_stacked = torch.stack(lm_head_weight_list)  # Shape: (130, 128)

    # Now replace the old list of weights with the stacked versions
    checkpoint['model']['transformer.wte.weight'] = wte_stacked
    checkpoint['model']['lm_head.weight'] = lm_head_stacked

    # Optionally, delete the old entries to avoid confusion
    for i in range(130):
        del checkpoint['model']['transformer.wte.weights.{}'.format(i)]
        del checkpoint['model']['lm_head.weights.{}'.format(i)]

    # Save the updated checkpoint
    fixed_ckpt_path = f"{out_path}/ckpt.pt"
    torch.save(checkpoint, fixed_ckpt_path)

    print(f"Fixed checkpoint saved to {fixed_ckpt_path}")

In [32]:
for margin in [1.5, 1.4, 1.3, 1.2, 1.1, 0.9, 0.8, 0.7, 0.6, 0.5]:
    model_name = f'proposed_margin_{margin}_separated_embeddings'
    args = {
        "out_dir": f"out/{model_name}",
        "model_type": "proposed",
        "margin": margin,
        "separated_embeddings": "True"
    }
    run_training(args)
    print(f'done for margin={margin}')
    fix_ckpt(model_name)

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

iter 7800: loss 1.3370, time 95.11ms, mfu 0.01%
iter 7900: loss 1.2700, time 124.03ms, mfu 0.01%
step 8000: train loss 1.7846, val loss 1.9441
iter 8000: loss 1.3367, time 1126.23ms, mfu 0.01%
done for margin=1.5


/tmp/ipykernel_341/2403027348.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location='cpu')


Fixed checkpoint saved to out/proposed_margin_1.5_separated_embeddings/ckpt.pt
Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # l

iter 7400: loss 1.3326, time 126.36ms, mfu 0.01%
step 7500: train loss 1.8321, val loss 1.9778
saving checkpoint to out/proposed_margin_1.4_separated_embeddings
iter 7500: loss 1.3059, time 1711.94ms, mfu 0.01%
iter 7600: loss 1.2578, time 105.27ms, mfu 0.01%
iter 7700: loss 1.2182, time 116.43ms, mfu 0.01%
step 7750: train loss 1.8500, val loss 1.9815
iter 7800: loss 1.3406, time 79.21ms, mfu 0.01%
iter 7900: loss 1.2593, time 74.34ms, mfu 0.01%
step 8000: train loss 1.8291, val loss 1.9898
iter 8000: loss 1.3165, time 1366.14ms, mfu 0.01%
done for margin=1.4


...

/tmp/ipykernel_341/2403027348.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location='cpu')


Fixed checkpoint saved to out/proposed_margin_0.6_separated_embeddings/ckpt.pt
Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # l

saving checkpoint to out/proposed_margin_0.5_separated_embeddings
iter 7800: loss 1.1913, time 277.35ms, mfu 0.01%
iter 7900: loss 1.1317, time 245.38ms, mfu 0.01%
step 8000: train loss 2.8286, val loss 2.9164
iter 8000: loss 1.1740, time 1246.93ms, mfu 0.01%
done for margin=0.5


/tmp/ipykernel_341/2403027348.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location='cpu')


Fixed checkpoint saved to out/proposed_margin_0.5_separated_embeddings/ckpt.pt


In [33]:
for margin in [4.0, 8.0]:
    args = {
        "out_dir": f"out/proposed_margin_{margin}_wo_weight_tying",
        "model_type": "proposed",
        "margin": margin,
    "weight_tying": "False"
        
    }
    run_training(args)
    print(f'done for margin={margin}')

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 8000
lr_decay_iters = 8000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of to

iter 7800: loss 1.3660, time 55.94ms, mfu 0.01%
iter 7900: loss 1.3607, time 126.62ms, mfu 0.01%
step 8000: train loss 1.4442, val loss 1.6123
iter 8000: loss 1.3152, time 2183.78ms, mfu 0.01%
done for margin=4.0
Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 100 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 12
block_size = 64 # context of up to 64 previous characters

# baby GPT model :)
n_layer = 4
n_head = 4
n_embd = 128
dropout = 0.0

learning_rate = 1e-3 # with 

saving checkpoint to out/proposed_margin_8.0_wo_weight_tying
iter 7500: loss 1.4120, time 1164.92ms, mfu 0.01%
iter 7600: loss 1.4322, time 128.99ms, mfu 0.01%
iter 7700: loss 1.4753, time 163.03ms, mfu 0.01%
step 7750: train loss 1.4090, val loss 1.6584
iter 7800: loss 1.3692, time 762.33ms, mfu 0.01%
iter 7900: loss 1.3797, time 209.39ms, mfu 0.01%
step 8000: train loss 1.4265, val loss 1.6138
iter 8000: loss 1.3417, time 2978.03ms, mfu 0.01%
done for margin=8.0


In [ ]:
# TODO
# for margin in [0.75, 0.55, 0.45, 0.4, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1, 0.05]:
#     model_name = f'proposed_margin_{margin}_separated_embeddings'
#     args = {
#         "out_dir": f"out/{model_name}",
#         "model_type": "proposed",
#         "margin": margin,
#         "separated_embeddings": "True"
#     }
#     run_training(args)
#     print(f'done for margin={margin}')
#     fix_ckpt(model_name)